In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Chrome, ChromeOptions
from bs4 import BeautifulSoup
from PIL import Image

import csv

# Lancer le navigateur web et charger la page web
driver = webdriver.Chrome()
driver.get("https://ipindiaservices.gov.in/PublicSearch/")

# Attendre jusqu'à ce que l'élément avec l'ID "TextField7" soit visible sur la page web
wait = WebDriverWait(driver, 10)
input_field_1 = wait.until(EC.visibility_of_element_located((By.ID, "TextField7")))
input_field_2 = wait.until(EC.visibility_of_element_located((By.ID, "FromDate")))
input_field_3 = wait.until(EC.visibility_of_element_located((By.ID, "ToDate")))

# Trouver le champ de saisie et y entrer la valeur "INDIA"
input_field_2.clear()
input_field_2.send_keys("01/01/2010")
input_field_3.clear()
input_field_3.send_keys("04/01/2023")
input_field_1.clear()
input_field_1.send_keys("INDIA")

# Descendre jusqu'en bas de la page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

# Zoomer sur la partie gauche de la page
driver.execute_script("document.querySelector('.container').style.width='200%'")

alphanumeric_text = input("Enter Captcha : ")

# Attendre que le champ de texte soit visible
text_input = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "CaptchaText"))
)

# Remplir le champ de texte avec "Captcha"
text_input.send_keys(alphanumeric_text)

# trouver l'élément du bouton "Search" et le cliquer
submit_button = driver.find_element(By.CSS_SELECTOR, "input[name='submit'][value='Search']")
submit_button.click()

# Ouvre le fichier "data.csv" en mode ajout
filename = 'data.csv'
with open(filename, 'a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    #writer.writerow(['Invention Title', 'Publication Number', 'Publication Date', 'Publication Type',
    #  ' Application Number', 'Application Filing Date', 'Priority Number', 'Priority Country', 
    # 'Priority Date', 'Field Of Invention', 'Classification (IPC)','Applicant', 'Inventor'])
    
    # Boucle sur toutes les pages de résultats
    while True:
        # Trouve tous les boutons ayant le nom "ApplicationNumber"
        buttons = driver.find_elements(By.NAME, "ApplicationNumber")

        # Pour chaque bouton, clique dessus pour ouvrir le lien dans un nouvel onglet
        for button in buttons:
            button.click()
            driver.switch_to.window(driver.window_handles[-1])  # Change de fenêtre pour le nouvel onglet

            # Traitement du nouvel onglet
            title = driver.title
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            data = {}
            table = soup.find('table', class_='table-striped')
            rows = table.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 2:
                    key = cells[0].text.strip()
                    value = cells[1].text.strip()
                    data[key] = value
                
            # Enregistrer les données dans le fichier CSV
            inventor_table = soup.find('td', text='Inventor').find_next('table')
            inventor_rows = inventor_table.find_all('tr')[1:]  # Exclude the header row

            inventors = []
            for row in inventor_rows:
                name, address, country, nationality = row.find_all('td')
                inventor = {
                    'Name': name.text.strip(),
                    'Address': address.text.strip(),
                    'Country': country.text.strip(),
                    'Nationality': nationality.text.strip()
                }
                inventors.append(inventor)

            applicant_table = soup.find('td', text='Applicant').find_next('table')
            applicant_rows = applicant_table.find_all('tr')[1:]  # Exclude the header row

            applicants = []
            for row in applicant_rows:
                name, address, country, nationality = row.find_all('td')
                applicant = {
                    'Name': name.text.strip(),
                    'Address': address.text.strip(),
                    'Country': country.text.strip(),
                    'Nationality': nationality.text.strip()
                }
                applicants.append(applicant)
                
            writer.writerow([value, applicants, inventors])

            try:
                driver.close()
            except:
                pass
            driver.switch_to.window(driver.window_handles[0])

        # Trouve le bouton "Next" s'il est disponible
        next_button = None
        try:
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "next")))
        except:
            break

        # Clique sur le bouton "Next" pour passer à la page suivante
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        driver.execute_script("arguments[0].click();", next_button)

# Ferme le driver de Chrome
file.close()
driver.quit()

Enter Captcha :  d5axa


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00CA8893+48451]
	(No symbol) [0x00C3B8A1]
	(No symbol) [0x00B45058]
	(No symbol) [0x00B2D073]
	(No symbol) [0x00B8DEBB]
	(No symbol) [0x00B9BFD3]
	(No symbol) [0x00B8A0B6]
	(No symbol) [0x00B67E08]
	(No symbol) [0x00B68F2D]
	GetHandleVerifier [0x00F08E3A+2540266]
	GetHandleVerifier [0x00F48959+2801161]
	GetHandleVerifier [0x00F4295C+2776588]
	GetHandleVerifier [0x00D32280+612144]
	(No symbol) [0x00C44F6C]
	(No symbol) [0x00C411D8]
	(No symbol) [0x00C412BB]
	(No symbol) [0x00C34857]
	BaseThreadInitThunk [0x75F67D59+25]
	RtlInitializeExceptionChain [0x77B6B74B+107]
	RtlClearBits [0x77B6B6CF+191]
